Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product

# Read data

In [2]:
run_on_kaggle = False

if run_on_kaggle:
    sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
else:
    sales = pd.read_csv('../data/sales_train.csv')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({"item_cnt_day": "sum"}).rename(columns={"item_cnt_day": "target"})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [15]:
all_data

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334300
141495,0,27,0,0.0,0.334300
144968,0,28,0,0.0,0.334300
142661,0,29,0,0.0,0.334300
138947,0,32,0,6.0,0.334300
...,...,...,...,...,...
10768834,59,22162,33,0.0,1.560268
10769024,59,22163,33,0.0,0.585938
10769690,59,22164,33,0.0,1.238693
10771216,59,22166,33,0.0,0.296170


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [4]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621644


#### Method 2

In [5]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621644


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [6]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold

# Split your data in 5 folds with sklearn.model_selection.KFold with shuffle=False argument.
kf = KFold(n_splits=5, shuffle=False)

# Iterate through folds...
for train_index, test_index in kf.split(all_data):
    print("TRAIN:", train_index, "TEST:", test_index)
    all_data_train, all_data_test = all_data.iloc[train_index], all_data.copy().iloc[test_index]
    
    # use all but the current fold (=test) to calculate mean target for each level item_id...
    item_id_target_mean = all_data_train.groupby('item_id').target.mean()
    
    # encode the test data 
    all_data_test['item_target_enc'] = all_data_test['item_id'].map(item_id_target_mean)
    
    # ... and fill the current fold
    all_data.iloc[test_index] = all_data_test
    
# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

TRAIN: [ 2182770  2182771  2182772 ... 10913847 10913848 10913849] TEST: [      0       1       2 ... 2182767 2182768 2182769]
TRAIN: [       0        1        2 ... 10913847 10913848 10913849] TEST: [2182770 2182771 2182772 ... 4365537 4365538 4365539]
TRAIN: [       0        1        2 ... 10913847 10913848 10913849] TEST: [4365540 4365541 4365542 ... 6548307 6548308 6548309]
TRAIN: [       0        1        2 ... 10913847 10913848 10913849] TEST: [6548310 6548311 6548312 ... 8731077 8731078 8731079]
TRAIN: [      0       1       2 ... 8731077 8731078 8731079] TEST: [ 8731080  8731081  8731082 ... 10913847 10913848 10913849]


In [7]:
encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4164590712798667


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [8]:
# YOUR CODE GOES HERE
# Calculate sum of the target values using all the objects.
total_sum_per_item = all_data.groupby('item_id')['target'].transform('sum')
n_objects_per_item = all_data.groupby('item_id')['target'].transform('count')

# Then subtract the target of the given object and divide the resulting value by n_objects - 1.
all_data['item_target_enc'] = (total_sum_per_item - all_data["target"]) / (n_objects_per_item - 1)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['item_target_enc'].values

In [9]:
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.48038483112929026


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [10]:
# YOUR CODE GOES HERE
globalmean = 0.3343
alpha = 100

mean_per_item = all_data.groupby('item_id')['target'].transform('mean')
n_rows_per_item = all_data.groupby('item_id')['target'].transform('count')

# formula: (mean(target) * nrows + globalmean * alpha) / (nrows + alpha)
all_data['item_target_enc'] = (mean_per_item * n_rows_per_item + globalmean * alpha) / (n_rows_per_item + alpha)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['item_target_enc'].values

In [11]:
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.48181987970971224


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [12]:
# YOUR CODE GOES HERE
cum_sum = all_data.groupby("item_id")["target"].cumsum() - all_data["target"]
cum_count =  all_data.groupby("item_id").cumcount()
all_data['item_target_enc'] = cum_sum / cum_count

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['item_target_enc'].values

In [14]:
encoded_feature

array([0.3343    , 0.3343    , 0.3343    , ..., 1.23869347, 0.29617021,
       1.08196721])

In [13]:
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081527
